# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [9]:
!pip install kaggle
!pip install kaggle-cli

In [10]:
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.automl import AutoMLConfig
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
from azureml.data.dataset_factory import TabularDatasetFactory
from sklearn.model_selection import train_test_split
from trainCovid19Dataset import clean_data
import os
import shutil

## Initialize Workspace

Create a workspace, if it doesn't exist, using the AzureML SDK

In [11]:
ws = Workspace.from_config()
    
ws.get_details()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: quick-starts-ws-140146
Azure region: southcentralus
Subscription id: cdbe0b43-92a0-4715-838a-f2648cc7ad21
Resource group: aml-quickstarts-140146


## Cluster

Get cluster if it exists else create one

In [12]:
amlcompute_cluster_name = "Covid19Cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
    print(compute_target.get_status().serialize())
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
{'currentNodeCount': 4, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 4, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-03-10T12:19:22.911000+00:00', 'errors': None, 'creationTime': '2021-03-10T10:21:50.752146+00:00', 'modifiedTime': '2021-03-10T10:22:06.565764+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

In [13]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
found = False
key = "Covid19Dataset"
description_text = "Covid19 Vaccination DataSet from Kaggle"
datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./kaggle/country_vaccinations.csv'],
                       target_path ='train-dataset/tabular/',
                       overwrite = True,
                       show_progress = True)
if key in ws.datasets.keys(): 
    found = True
    dataset = ws.datasets[key] 

df = dataset.to_pandas_dataframe().fillna(0)
df.describe()

Uploading an estimated of 1 files
Uploading ./kaggle/country_vaccinations.csv
Uploaded ./kaggle/country_vaccinations.csv, 1 files out of an estimated total of 1
Uploaded 1 files


,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million
count,5.689000e+03,5.689000e+03,5.689000e+03,5.689000e+03,5.689000e+03,5689.000000,5689.000000,5689.000000,5689.000000
mean,1.318467e+06,9.688013e+05,2.472286e+05,4.369882e+04,5.619554e+04,5.279773,3.686785,1.076433,2462.084901
std,5.744025e+06,4.136228e+06,1.609410e+06,1.773927e+05,1.840107e+05,12.839767,8.609800,4.175587,4325.232412
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.001000e+03,0.000000,0.000000,0.000000,301.000000
50%,2.606000e+04,1.052100e+04,0.000000e+00,3.110000e+02,5.767000e+03,0.480000,0.170000,0.000000,1090.000000
75%,4.633830e+05,2.940590e+05,2.893700e+04,1.525700e+04,2.729100e+04,4.830000,3.290000,0.640000,2563.000000
max,9.369260e+07,6.108853e+07,3.210206e+07,2.904229e+06,2.169981e+06,131.310000,84.390000,46.920000,54264.000000


In [14]:
ws = Workspace.from_config()
experiment_name = 'Covid19HyperDrExperiment'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [15]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor = 0.2, evaluation_interval=2, delay_evaluation=5)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling( 
    {
        '--C': choice(0.01, 0.1, 1, 10, 100), 
        '--max_iter': choice(25, 50, 100,150)
    }
)
if "training" not in os.listdir():
    os.mkdir("./training")
script_folder = './training/'    
os.makedirs(script_folder, exist_ok=True)
shutil.copy('trainCovid19Dataset.py', script_folder)

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory=script_folder, compute_target= amlcompute_cluster_name, entry_script='trainCovid19Dataset.py')

hyperdrive_run_config = HyperDriveConfig(
                                   hyperparameter_sampling = param_sampling,
                                   primary_metric_name = 'Accuracy',
                                   primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                   max_total_runs=30,
                                   max_concurrent_runs = 3,
                                   policy = early_termination_policy,
                                   estimator = estimator)

In [16]:
#TODO: Submit your experiment
tag = {"Covid19": "Capstone project: Covid19 HyperDrive Experiment"}
remote_run = experiment.submit(hyperdrive_run_config,tags=tag, show_output=True)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [22]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_93b4d8d3-f8cf-4afe-ac7a-9044d89cc3ec
Web View: https://ml.azure.com/experiments/Covid19HyperDrExperiment/runs/HD_93b4d8d3-f8cf-4afe-ac7a-9044d89cc3ec?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-140146/workspaces/quick-starts-ws-140146

Execution Summary
RunId: HD_93b4d8d3-f8cf-4afe-ac7a-9044d89cc3ec
Web View: https://ml.azure.com/experiments/Covid19HyperDrExperiment/runs/HD_93b4d8d3-f8cf-4afe-ac7a-9044d89cc3ec?wsid=/subscriptions/cdbe0b43-92a0-4715-838a-f2648cc7ad21/resourcegroups/aml-quickstarts-140146/workspaces/quick-starts-ws-140146

Warnings:
{
  "error": {
    "code": "UserError",
    "severity": null,
    "message": "User errors were found in at least one of the child runs.",
    "messageFormat": null,
    "messageParameters": {},
    "referenceCode": null,
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": null
  },
  "correlation": null,
  "environment": null,
  "location

{'runId': 'HD_93b4d8d3-f8cf-4afe-ac7a-9044d89cc3ec',
 'target': 'Covid19Cluster',
 'status': 'Canceled',
 'startTimeUtc': '2021-03-10T13:05:59.260036Z',
 'endTimeUtc': '2021-03-10T13:25:28.007105Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'messageParameters': {},
   'details': []},
  'time': '0001-01-01T00:00:00.000Z'},
 'warnings': [{'source': 'SecondaryError',
   'message': '{\n  "error": {\n    "code": "UserError",\n    "severity": null,\n    "message": "User errors were found in at least one of the child runs.",\n    "messageFormat": null,\n    "messageParameters": {},\n    "referenceCode": null,\n    "detailsUri": null,\n    "target": null,\n    "details": [],\n    "innerError": null,\n    "debugInfo": null\n  },\n  "correlation": null,\n  "environment": null,\n  "location": null,\n  "time": "0001-01-01T00:00:00+00:00",\n  "componentName": null\n}'}],
 'properties': {'primary_metric_config': '{"name": 

In [21]:
import joblib
# Get your best run and save the model from that run.
best_run_HyperDr = remote_run.get_best_run_by_primary_metric()
# joblib.dump(best_run_sklearn, 'training/sklearn_bankmarketing_model.joblib')
SKLearn_Model = best_run_HyperDr.register_model(model_name="sklearnBankmarketingModel", model_path='outputs/model.joblib')
# Get the metrics of the bestselected run
best_run_metrics = best_run_HyperDr.get_metrics()
# Show the Accuracy of that run
print('Best Accuracy: {}'.format(best_run_metrics['Accuracy']))
best_run_HyperDr

AttributeError: 'NoneType' object has no attribute 'register_model'

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service